In [9]:
import pandas as pd
import numpy as np

df=pd.read_csv("Indian-Law.csv")
df.head(5)

,Instruction,Response
0,What is the difference between a petition and ...,A petition is a formal request submitted to a ...
1,When should a writ petition be filed in India?,A writ petition in India should be filed when ...
2,What is the procedure for filing a plaint in a...,To file a plaint in a civil case in Indiayou m...
3,What are the common reliefs sought through a p...,Public interest litigation (PIL) petitions in ...
4,Can a plaint be amended after it has been file...,Yesa plaint can be amended in a civil case in ...


In [8]:
import openai
import chromadb

In [21]:
# !pip install chroma-migrate

In [22]:
# !pip install --upgrade pip setuptools wheel


In [23]:
# from sentence_transformers import SentenceTransformer


# model = SentenceTransformer('all-MiniLM-L6-v2')


# def get_embedding(text):
#     return model.encode(str(text)).tolist()


# df['embedding'] = df['Instruction'].apply(get_embedding)

# df.head(5)

# df.to_csv('Indian-Law_embeddings.csv', index=False)


In [4]:
# df=pd.read_csv("Indian-Law_embedding.csv")
# df.head(5)

,Instruction,Response,embedding
0,What is the difference between a petition and ...,A petition is a formal request submitted to a ...,"[-0.028258008882403374, 0.07087016850709915, -..."
1,When should a writ petition be filed in India?,A writ petition in India should be filed when ...,"[0.014730489812791348, -0.010633070021867752, ..."
2,What is the procedure for filing a plaint in a...,To file a plaint in a civil case in Indiayou m...,"[-0.04823501408100128, 0.11056535691022873, -0..."
3,What are the common reliefs sought through a p...,Public interest litigation (PIL) petitions in ...,"[-0.02925526164472103, 0.0396060012280941, -0...."
4,Can a plaint be amended after it has been file...,Yesa plaint can be amended in a civil case in ...,"[-0.07011803984642029, 0.11076454818248749, 0...."


In [25]:
client = chromadb.PersistentClient(path="indian_law_kp_work")

collection = client.create_collection("indian_law_kp_work")


In [26]:

for index, row in df.iterrows():
   
    embedding = eval(row['embedding']) if isinstance(row['embedding'], str) else row['embedding']
    
    collection.add(
        ids=str(index),
        embeddings=[embedding],  
        documents=[row['Response']],
        metadatas={"instruction": row['Instruction']}
    )

print("Data stored in ChromaDB successfully.")


Data stored in ChromaDB successfully.


In [25]:
client = chromadb.PersistentClient(path="indian_law_kp_work")
collection=client.get_or_create_collection("indian_law_kp_work")

query=" article 51 of indian constitution"
results=collection.query(query_texts=[query],n_results=1)
print(results)

{'ids': [['153']], 'distances': [[4.553018118774732e-13]], 'metadatas': [[{'instruction': 'Article 51 of Indian Constitution'}]], 'embeddings': None, 'documents': [['Promotion of international peace and security The State shall endeavour to\n(a) promote international peace and security;\n(b) maintain just and honourable relations between nations;\n(c) foster respect for international law and treaty obligations in the dealings of organised peoples with one another; and encourage settlement of international disputes by arbitration PART IVA FUNDAMENTAL DUTIES"\nArticle 51A of Indian Constitution,"Fundamental duties It shall be the duty of every citizen of India (a) to abide by the Constitution and respect its ideals and institutions, the national Flag and the National Anthem;\n(b) to cherish and follow the noble ideals which inspired our national struggle for freedom;\n(c) to uphold and protect the sovereignty, unity and integrity of India;\n(d) to defend the country and render national s

In [10]:
from sentence_transformers import SentenceTransformer

# Load BGE small model
bge_model = SentenceTransformer('BAAI/bge-small-en-v1.5')

def get_bge_embedding(text):
    return bge_model.encode(str(text)).tolist()

# Generate BGE embeddings
df['bge_embedding'] = df['Instruction'].apply(get_bge_embedding)

/home/kushagra/Documents/code/AI/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
df.head()

,Instruction,Response,bge_embedding
0,What is the difference between a petition and ...,A petition is a formal request submitted to a ...,"[-0.05384553596377373, 0.02437661774456501, -0..."
1,When should a writ petition be filed in India?,A writ petition in India should be filed when ...,"[-0.09163694828748703, -0.04776277020573616, 0..."
2,What is the procedure for filing a plaint in a...,To file a plaint in a civil case in Indiayou m...,"[-0.03673609718680382, 0.017968948930501938, -..."
3,What are the common reliefs sought through a p...,Public interest litigation (PIL) petitions in ...,"[-0.07224810123443604, -0.0032734533306211233,..."
4,Can a plaint be amended after it has been file...,Yesa plaint can be amended in a civil case in ...,"[-0.06682822853326797, -0.02113156206905842, 0..."


In [13]:
bge_client = chromadb.PersistentClient(path="indian_law_bge_work_1")

bge_collection = bge_client.create_collection("indian_law_bge_work_1")

# Store BGE embeddings in the new collection


In [14]:
for index, row in df.iterrows():

    bge_embedding = eval(row['bge_embedding']) if isinstance(row['bge_embedding'], str) else row['bge_embedding']


    instruction_str = str(row['Instruction'])
    response_str = str(row['Response'])

    bge_collection.add(
        ids=str(index),
        embeddings=[bge_embedding],  
        documents=["question:" + instruction_str + "\n" + "answer:" + response_str],
        metadatas={"instruction": instruction_str}
    )

print("BGE Data stored in ChromaDB successfully.")


BGE Data stored in ChromaDB successfully.


In [23]:
client = chromadb.PersistentClient(path="indian_law_bge_work_1")
collection=client.get_or_create_collection("indian_law_bge_work_1")

query=" article 51 "
results=collection.query(query_texts=[query],n_results=4)
print(results['metadatas'])

[[{'instruction': 'Article 211 of Indian Constitution'}, {'instruction': 'Article 51 of Indian Constitution'}, {'instruction': 'What are the objectives of Article 51 of the Indian Constitution?'}, {'instruction': 'What are the objectives of Article 51 of the Indian Constitution?'}]]


In [2]:
!pip install gdown